In [ ]:
import torch
from midi_model import MidiModel

In [ ]:
mh = MidiModel()
mh.init_data_dir()
mh.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mh.epochs = 10
mh.batch_size = 16
mh.learning_rate = 0.001

In [ ]:
mh.load_train_data()

In [ ]:
mh.load_train_notes(0, 10)

In [ ]:
mh.create_model()

In [ ]:
mh.train_model()

In [ ]:
input_notes = mh.midi.get_notes(mh.midi.files[10])
num_predictions = 50
seq_length = 5
temperature = 1.0

generated_notes, out_pm = mh.generate_notes(input_notes, num_predictions, seq_length, temperature)

mh.midi.display_midi("../data/generated/output.mid")

In [ ]:
mh.midi.display_audio(out_pm)